In [1]:
from encoders import EncoderResNet18, encoderCNN, bn_relu
from decoders import DecoderResNet18

from src.CVAE import CVAE
from DataLoader import data_loader


In [2]:
train_x, test_x, val_x, train_y, test_y, val_y, train_y_one_hot, test_y_one_hot, val_y_one_hot, input_shape, category_count, labels = data_loader(name='histo', root_folder='/home/PERSONALE/nicolas.derus2/HistoDL/datasets/')

<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
encoded_dim = 128
category_count = 2
input_shape = (64, 64, 3)

In [ ]:
encoder = EncoderResNet18(encoded_dim = encoded_dim)
encoder = encoder.model(input_shape=(64, 64, 5))
encoder

In [ ]:
decoder = DecoderResNet18( encoded_dim = encoded_dim)
decoder = decoder.model(input_shape=(encoded_dim + category_count,))
decoder

In [ ]:
#kl_coefficient = encoded_dim / (input_shape[0] * input_shape[1] * input_shape[2]) # from b vae paper, use beta = encoded_dimension / pixel_dimension i.e. -> 0.068
kl_coefficient = .65
print('kl coefficient: {:.3f}'.format(kl_coefficient))
cvae = CVAE(encoder, decoder, kl_coefficient, input_shape)
cvae.built = True
cvae.summary()

In [ ]:
#if available:
# distribute_strategy on TPU